# Práctica 1

Gonzalo Monedero - Marcos Molina - Nicolás Salgado - Iñigo Vázquez

## Preparación del entorno

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras as ker

## Variables a definir

In [ ]:
"""
MatrizPesos: Número de pesos de la matriz de pesos 
NPOB: Número de individuos de la población 
NGEN: Número de generaciones (ciclo completo sobre todos los individuos) 
Pm: Probabilidad de mutación: Probabilidad de que un gen dado mute 
NRES: Cada cuántas generaciones se imprime un resumen de la evolución del proceso 
NSAMPLE: Cada qué número de generaciones se saca un muestreo de la población 
"""

matriz_pesos = []
npob = 1 # NO TOCAR
ngen = 1 # TOCAR
pm = 1 # TOCAR
nres = 1 # TOCAR
nsample = 1 # TOCAR

## Preprocesamiento de datos

### Creación del dataset


In [4]:
iris_RAW = pd.read_csv('../Data/iris.data', sep=',')

## Funciones algoritmo genético

### Codificar

### Descodificar

### Función fit

# Bucle principal